<a href="https://colab.research.google.com/github/Mortal-24/chatbot-template-qmdqbrx1l3d/blob/main/Speech_emotion_recognition(SVC%2C_Random_Forest).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'toronto-emotional-speech-set-tess:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F316368%2F639622%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240708%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240708T122458Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4780687998e70b2e41ff93582f895dc7bfc00a4e50651f4372e2cbdfb8049159fdb7782d1419ee24c4919c5f43ac2f08c986397b58d9800229af73d298eb2b4e5b915e1a7b51a9773cbdc241eedcf79eadea61ec54ea4aee9de6d0646a53a0cd901aae1e6f835dc9db2a85e0b7d261f60dfddafe259aab135c72dbe8785d8ca9c69d912f03f08d86b3e0fbb4052fa840e3a27a3bffc1d597197abed0468985a2208eb35f39f37f36801cb2a5ce23265d899096a72ad1c3eeff06ed1d64f2ab28397b9dd4cf6f2801b89c294ca53fcefb64da951847fce6bbf21618c5fba1bfa07095496fee46d86414d0b1a8813808a929c801253f8b9d5a5019c3e6ac08617a'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Import Libraries**

In [ ]:
import pandas as pd
import numpy as np
import librosa
import matplotlib.pyplot as plt
import seaborn as sns
import os
import librosa.display
import IPython.display as ipd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.models import Sequential
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Define the folder containing the audio signals
root_directory = '/kaggle/input/toronto-emotional-speech-set-tess/TESS Toronto emotional speech set data'

# Define the function to preprocess and extract features from audio signals
def preprocess_and_extract_features(file_path):
    try:
        # Load the audio file
        audio_data, sample_rate = librosa.load(file_path, sr=None)

        # Apply preprocessing steps
        # Resample to a common sample rate (if necessary)
        target_sample_rate = 45000  # Target sample rate
        if sample_rate != target_sample_rate:
            audio_data = librosa.resample(audio_data, orig_sr=sample_rate, target_sr=target_sample_rate)

        # Normalize the amplitude of the audio signal
        audio_data /= np.max(np.abs(audio_data))

        # Feature extraction
        #Compute Mel-Frequency Cepstral Coefficients (MFCCs)
        mfccs = librosa.feature.mfcc(y=audio_data, sr=target_sample_rate, n_mfcc=40)

        return mfccs

    except Exception as e:
        print(f"Error processing file {file_path}: {str(e)}")
        return None

# **Iterate the audio files and apply feature extraction**

In [ ]:
# Iterate through each audio file in the folder
labels = []
mfcc_features = []

for folder in os.listdir(root_directory):
    folder_path = os.path.join(root_directory,folder)
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.wav'):
            file_path = os.path.join(folder_path, file_name)
            print(f'Processing file: {file_name}')

            # Preprocess and extract features from the audio signal
            features = preprocess_and_extract_features(file_path)
            features = np.mean(features.T,axis=0)
            mfcc_features.append(features)
            labels.append(folder)

# **Demo EDA**

In [ ]:
path = "/kaggle/input/toronto-emotional-speech-set-tess/TESS Toronto emotional speech set data/OAF_Fear/OAF_back_fear.wav"

plt.figure(figsize=(16,6))
data,s_r = librosa.load(path)
librosa.display.waveshow(data,sr=s_r,color='green')
plt.title("Time Domain")
plt.grid(True)
plt.show()

In [ ]:
#make  a dataframe
df = pd.DataFrame({
    'MFCC':mfcc_features,
    'Label': labels
})
df.shape

# **Shuffle the rows**

In [ ]:
new_df = df.sample(2800)
new_df['Label'].value_counts()

# **Categorical Distribution**

In [ ]:
import seaborn as sns
plt.figure(figsize=(16,6))
sns.countplot(new_df,x='Label')
plt.tight_layout()
plt.title("Categorical Distribution")
plt.show()

In [ ]:
#Label Encoding
lbl = LabelEncoder()
new_df['Label'] = lbl.fit_transform(new_df['Label'])
new_df.head()

In [ ]:
# separate the label and feature

X = np.array(new_df['MFCC'].tolist())
y = np.array(new_df['Label'].tolist())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# **Support vector machine**

In [ ]:
sv = SVC()
sv.fit(X_train,y_train)

In [ ]:
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay,accuracy_score,recall_score,precision_score
import matplotlib.pyplot as plt

y_pred_sv = sv.predict(X_test)

print(f"Accuracy from svc:{accuracy_score(y_test,y_pred_sv)*100:.4f} %")
print(f"recall_score: {precision_score(y_test,y_pred_sv,average='weighted')*100:.4f} %")

cnf_sv = confusion_matrix(y_test,y_pred_sv)
ax = ConfusionMatrixDisplay(cnf_sv).plot()
plt.title("Confusion Matrix")
plt.show()

# **RandomForest**

In [ ]:
rnf = RandomForestClassifier()
rnf.fit(X_train,y_train)
print(f"{rnf.score(X_test,y_test)*100:.4f} %")

In [ ]:
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
import matplotlib.pyplot as plt

y_pred = rnf.predict(X_test)
cnf = confusion_matrix(y_test,y_pred)

ax = ConfusionMatrixDisplay(cnf,display_labels=lbl.classes_).plot()
plt.xticks(rotation=90)
plt.show()

# **ANN**

In [ ]:
model = Sequential()
model.add(Dense(256,activation='relu',input_shape=(40,)))
model.add(Dense(128,activation='relu'))
model.add(Dense(14,activation='softmax'))

model.compile(optimizer='adam',metrics=['accuracy'],loss='sparse_categorical_crossentropy')
model.fit(X_train,y_train,epochs=10,batch_size=32,validation_data=(X_test,y_test))

In [ ]:
print(f"accuracy: {model.evaluate(X_test,y_test)[1]*100:.4f}%")

In [ ]:
y_pred_ann = np.argmax((model.predict(X_test)>0.5).astype(int),axis=1)

cnf_ann = confusion_matrix(y_test,y_pred_ann)
ax_ann = ConfusionMatrixDisplay(cnf_ann,display_labels=lbl.classes_).plot()
plt.title("Confusion Matrix")
plt.xticks(rotation=90)
plt.show()